### 랭그래프 과제
라이브러리 선언

In [42]:
from langchain_core.documents import Document
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from pprint import pprint
import os

load_dotenv()

True

In [34]:
# 동물 데이터 베이스 (딕셔너리) 정의
animal_facts = {
    "강아지":"강아지는 인간과 가장 친한 반려동물 중 하나입니다. 충성심이 강하고 활발 합니다.",
    "고양이":"고양이는 독립적인 성격을 가진 반려동물로, 꺠끗하고 조용한 성격을 가졌습니다.",
    "코끼리":"코끼리는 육상 동물 중 가장 큰 동물로, 매우 높은 지능과 사회성을 지닙니다.",
    "기린":"기린은 목이 매우 길며, 주로 아프리카 사바나에서 서식합니다.",
}

In [ ]:
%pip install langchain-community duckduckgo-search

   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   -------------------- ------------------- 2.1/4.0 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 11.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ----------------------------- ---------- 2.4/3.1 MB 12.2 MB/s eta 0:00:01
   ---------------------------------------- 3.1/3.1 MB 10.9 MB/s eta 0:00:00

   ------------- -------------------------- 1/3 [lxml]
   ------------- -------------------------- 1/3 [lxml]
   ---------------------------------------- 3/3 [duckduckgo-search]




[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
today = datetime.today().strftime("%Y-%m-%d")
prompt = ChatPromptTemplate([
    ("system", 
    "너는 동물 정보를 알려주는 어시스턴트야. "
    "모든 질문에 답변하기 전에 반드시 search_animal 또는 웹검색 도구를 사용해야 한다. "
    "자신의 지식만으로 답변하지 말고, 도구의 결과를 참고해서만 답하라."
    f"항상 마지막엔 {today}를 반환해서 타임라인을 만들어야한다."),
    ("human","{user_input}"),
    ("placeholder","{message}")
])

In [ ]:
# 덕덕고로 웹서치
from langchain_community.tools import DuckDuckGoSearchResults

web_search = DuckDuckGoSearchResults(max_results=2)
search_results = web_search.invoke()

for result in search_results:
    print(result)
    print('-'*50)

NameError: name 'search_results' is not defined

In [37]:
# 동물 정보 반환 tool 셋팅
@tool
def search_animal(query: str) -> list[Document]:
    """
    이름에 해당하는 동물 정보를 간단히 반환합니다.
    """
    if query in animal_facts:
        return [Document(page_content=animal_facts[query])]
    return [Document(page_content="해당 동물의 정보를 찾을 수 없습니다.")]

In [43]:
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools=[search_animal])
llm_chain = prompt | llm_with_tools

ai_msg = llm_with_tools.invoke('고양이에 대해 알려줘')

pprint(ai_msg)
pprint('-'*100)

pprint(ai_msg.content)
pprint('-'*100)

pprint(ai_msg.tool_calls)           # llm이 tool을 호출할지 말지를 판단할 수 있음. 
pprint('-'*100)


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bPYdOti1dEPU8XAtMhxJQ6xc', 'function': {'arguments': '{"query":"고양이"}', 'name': 'search_animal'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 57, 'total_tokens': 74, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--44d3b760-2c3b-4867-9182-cc614f995646-0', tool_calls=[{'name': 'search_animal', 'args': {'query': '고양이'}, 'id': 'call_bPYdOti1dEPU8XAtMhxJQ6xc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, 'output_tokens': 17, 'total_tokens': 74})
'----------------------------------------------------------------------------------------------------'


In [44]:
@chain
def web_search_chain(user_input: str):
    input_ = {"user_input":user_input}
    ai_msg = llm_chain.invoke(input_)
    tool_msgs = web_search.batch(ai_msg.tool_calls)
    print(f"tool_msgs: \n{tool_msgs}")
    print('-'*100)
    return llm_chain.invoke({**input_, "message":[ai_msg, *tool_msgs]})

In [41]:
# 실행
question = "고양이에 대해 알려줘"
respose = web_search_chain.invoke("고양이에 대해 알려줘?")
print(f"최종응답: \n {respose.content}")

c:\Users\scv31\AppData\Local\pypoetry\Cache\virtualenvs\langgraph-works-VZ1_LVEb-py3.11\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
c:\Users\scv31\AppData\Local\pypoetry\Cache\virtualenvs\langgraph-works-VZ1_LVEb-py3.11\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


tool_msgs: 
[ToolMessage(content='snippet: Aug 8, 2025 · 마지막으로, 저연령층 사이에서 컬트적인 인기를 끈 공포 게임 ‘파피 플레이타임’에도 고양이 캐릭터가 등장한다. 바로 챕터 3의 메인 빌런이자 최종 보스인 ‘캣냅 (CatNap)’이다. 이 …, title: ‘나옹’, ‘아이루’, ‘모르가나’... 고양이의 날이면 생각 ..., link: https://dpg.danawa.com/news/view?boardSeq=60&listSeq=5862768, snippet: Aug 1, 2025 · 프렌치 감성 뷰티 브랜드 폴앤조 보떼(PAUL & JOE)가 고양이를 모티프로 한 시즌 한정 립밤 ‘골든캣 립밤 리미티드 에디션’을 오는 8월 1일 SSG에서 단독 선런칭한다. 이번 리미티드 …, title: 폴앤조 보떼, 황금 고양이 감성 담은 ‘골든캣 립밤 리미티드 ..., link: https://dpg.danawa.com/news/view?boardSeq=61&listSeq=5859059, snippet: Apr 17, 2023 · 로봇청소기 후기를 읽다 보면 간혹 괴담처럼 전해지는 이야기가 있다. ‘강아지가 간밤에 거실에 변을 봤는데, 예약 청소를 걸어둔 로봇청소기가 변을 그대로 밀고 지나가면서 온 집안이 …, title: 로봇청소기는 반려동물 대변과 토사물을 어떻게 청소할까?, link: https://dpg.danawa.com/news/view?boardSeq=67&listSeq=5321152&past=Y, snippet: Aug 14, 2025 · 실시간 3D 콘텐츠 제작 및 성장 플랫폼 유니티가 ‘17회 유니티 어워즈(17th Unity Awards)’의 후보작 모집을 시작한다고 14일 밝혔다. 유니티 어워즈는 창의성과 예술적·기술적 가치, …, title: “고양이 오마카세 뒤 이을 게임은?” 유니티, ‘17회 유니티 ..., link: https://dpg.danawa.com/news/view?boardSeq